In [1]:
from transformer_lens import HookedTransformer
import circuitsvis as cv
device = "cuda"
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


### L4 H7

Let us check whether L4H7 attends is more likely to attend to duplicate tokens (previous occurence of the destination token), or to attend to the destination token itself. 

Here, I give it a prompt where a sentence repeats itself. Looking at the attention patterns of L4H7, it seems that the highest amount of attention for this head is to the destination token itself and not to the previous occurence of the current token. This seems to refute Kissane et al's interpretation of L4H7 as "duplicate token" head.

In [2]:
gpt2_text = "Natural language processing tasks, such as question answering, machine translation, reading comprehension, and summarization, are typically approached with supervised learning on taskspecific datasets. Natural language processing tasks, such as question answering, machine translation, reading comprehension, and summarization, are typically approached with supervised learning on taskspecific datasets."
gpt2_tokens = model.to_tokens(gpt2_text)
print(gpt2_tokens.device)
gpt2_logits, gpt2_cache = model.run_with_cache(gpt2_tokens, remove_batch_dim=True)

cuda:0


In [3]:
layer = 4
attention_pattern = gpt2_cache["pattern", layer, "attn"]
gpt2_str_tokens = model.to_str_tokens(gpt2_text)

In [4]:
print(f"Layer {4} Head Attention Patterns:")
cv.attention.attention_patterns(tokens=gpt2_str_tokens, attention=attention_pattern)

Layer 4 Head Attention Patterns:


### L4 H2

To analyze L4H2, 

In [10]:
# Define the food recipes as strings
recipe_1 = """
Pancakes:
Ingredients:
- 1 cup all-purpose flour
- 2 tablespoons sugar
- 1 cup milk
- 1 large egg
- 2 tablespoons melted butter
- 1 teaspoon baking powder
- A pinch of salt

Instructions:
1. In a bowl, mix flour, sugar, baking powder, and salt.
2. In another bowl, whisk milk, egg, and melted butter.
3. Combine wet and dry ingredients.
4. Heat a non-stick pan over medium heat.
5. Pour 1/4 cup batter onto the pan.
6. Cook until bubbles form, then flip.
7. Serve warm with syrup.

"""

recipe_2 = """
Chocolate Cake:
Ingredients:
- 1 and 1/2 cups all-purpose flour
- 1 cup sugar
- 1/2 cup cocoa powder
- 1 teaspoon baking powder
- 1/2 teaspoon baking soda
- 1/2 cup vegetable oil
- 2 large eggs
- 1 cup milk
- 2 tablespoons vanilla extract

Instructions:
1. Preheat the oven to 350°F.
2. Mix flour, sugar, cocoa, baking powder, and soda.
3. Add eggs, milk, oil, and vanilla.
4. Beat until smooth.
5. Pour batter into a greased pan.
6. Bake for 30-35 minutes.
7. Cool before serving.

"""

recipe_3 = """
Tomato Soup:
Ingredients:
- 4 cups tomatoes, chopped
- 1/2 cup onion, chopped
- 2 tablespoons olive oil
- 2 cups vegetable broth
- 1 teaspoon sugar
- 1/2 cup cream
- 2 tablespoons basil, chopped

Instructions:
1. Heat olive oil in a pot.
2. Add onions and sauté until soft.
3. Add tomatoes, sugar, and broth.
4. Simmer for 20 minutes.
5. Puree the soup.
6. Stir in cream and basil.
7. Serve hot.

"""

recipe_4 = """
Garlic Bread:
Ingredients:
- 1 loaf French bread
- 1/2 cup butter, melted
- 4 cloves garlic, minced
- 2 tablespoons parsley, chopped
- 1/2 cup parmesan cheese, grated

Instructions:
1. Preheat the oven to 375°F.
2. Mix melted butter, garlic, and parsley.
3. Slice the bread and spread the mixture on each piece.
4. Sprinkle with parmesan cheese.
5. Bake for 10-12 minutes.
6. Serve warm.

"""

recipe_5 = """
Muffins:
Ingredients:
- 2 cups all-purpose flour
- 1/2 cup sugar
- 1/2 cup butter, melted
- 1/2 cup milk
- 1 large egg
- 2 tablespoons baking powder
- 1 teaspoon vanilla extract

Instructions:
1. Preheat oven to 400°F.
2. Mix flour, sugar, and baking powder.
3. In another bowl, combine milk, butter, egg, and vanilla.
4. Combine wet and dry ingredients.
5. Fill muffin tins 3/4 full.
6. Bake for 15-18 minutes.
7. Cool before serving.

"""

# Create the dataset of food recipes
recipes = [recipe_1, recipe_2, recipe_3, recipe_4, recipe_5]

In [27]:
news_1 = """
Tech Industry:
Silicon Valley, August 30, 2024 - Major tech companies have announced a joint initiative to improve data privacy across their platforms. The collaboration, called "Privacy First," aims to develop new encryption standards and reduce the amount of personal data collected by online services. Industry leaders praised the move as a significant step towards safeguarding user privacy in the digital age.
"""

news_2 = """
Health:
New York, August 30, 2024 - A new study published in the Journal of Medical Research reveals that regular exercise can significantly reduce the risk of chronic diseases, including heart disease and diabetes. The study followed over 10,000 participants for five years and found that those who engaged in at least 30 minutes of physical activity daily had a 40% lower risk of developing these conditions.
"""

news_3 = """
Environment:
Paris, August 30, 2024 - The French government has introduced new legislation aimed at reducing carbon emissions by 50% over the next decade. The plan includes incentives for renewable energy projects, stricter regulations on industrial emissions, and increased funding for public transportation. Environmental groups have welcomed the move but are calling for even more ambitious targets to combat climate change.
"""

news_4 = """
Economy:
Tokyo, August 30, 2024 - Japan's economy grew by 2.5% in the second quarter of 2024, driven by strong consumer spending and a resurgence in manufacturing. The latest figures from the Ministry of Finance show that the country is on track for its best annual growth rate in nearly a decade. Economists credit the government's stimulus measures and a weakening yen for boosting exports.
"""

news_5 = """
Science:
Geneva, August 30, 2024 - Scientists at CERN have made a groundbreaking discovery in particle physics. The team detected a new subatomic particle that could provide insights into the fundamental forces of the universe. The discovery was made using the Large Hadron Collider, and researchers believe it could lead to a deeper understanding of dark matter and the origins of the universe.
"""

In [26]:
gpt2_text = recipe_3[:300]
gpt2_tokens = model.to_tokens(gpt2_text)
print(gpt2_tokens.device)
gpt2_logits, gpt2_cache = model.run_with_cache(gpt2_tokens, remove_batch_dim=True)

layer = 4
attention_pattern = gpt2_cache["pattern", layer, "attn"]
gpt2_str_tokens = model.to_str_tokens(gpt2_text)

print(f"Layer {4} Head Attention Patterns:")
cv.attention.attention_patterns(tokens=gpt2_str_tokens, attention=attention_pattern)

cuda:0
Layer 4 Head Attention Patterns:


In [31]:
gpt2_text = news_4[:300]
gpt2_tokens = model.to_tokens(gpt2_text)
print(gpt2_tokens.device)
gpt2_logits, gpt2_cache = model.run_with_cache(gpt2_tokens, remove_batch_dim=True)

layer = 4
attention_pattern = gpt2_cache["pattern", layer, "attn"]
gpt2_str_tokens = model.to_str_tokens(gpt2_text)

print(f"Layer {4} Head Attention Patterns:")
cv.attention.attention_patterns(tokens=gpt2_str_tokens, attention=attention_pattern)

cuda:0
Layer 4 Head Attention Patterns:
